Import Libraries

In [74]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import math 
import cv2
import pytesseract
import re
from time import time
from decimal import ROUND_UP
from math import ceil
import pandas as pd

Get all the property addresses

In [83]:
# rightmove main URL
### initialise search ### 
#URL = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=POSTCODE%5E1553684&minBedrooms=2&maxPrice=700000&radius=1.0&index=72&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
URL = 'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=POSTCODE%5E1553684&minBedrooms=2&maxPrice=700000&radius=1.0&sortType=18&propertyTypes=detached%2Cflat%2Csemi-detached%2Cterraced&mustHave=&dontShow=&furnishTypes=&keywords=freehold'
page = requests.get(URL)

# parse search results card
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id='l-container')

# index for iterator (remember each page has 24 ads)
noOfResults = soup.find('div', attrs={'id': 'searchHeader'})
noOfResults = noOfResults.select('span')[0].text
noOfResults = ceil(int(noOfResults)/24) # round up to nearest page

### Start Loop ###
i = 0
all_ids = np.array([])
adLinks = np.array([])

print(noOfResults, " Pages available")

for x in range(noOfResults):
    URL = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=POSTCODE%5E1553684&minBedrooms=2&maxPrice=700000&radius=1.0&index=" + str(i) + "&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
    page = requests.get(URL)

    # parse search results card
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id='l-container')

    # Extract and store ids
    links = results.select('a')
    for card in links:
        ids = card.get('id')
        ids = ids.strip() if ids is not None else ''
        all_ids = np.append(all_ids, ids)

    # clean array and add links
    all_ids = all_ids[all_ids != '']
    cleanIds = np.char.strip(all_ids, 'prop')
    allLinks = np.char.add(cleanIds, '#/?channel=RES_BUY')
    allLinks = np.char.add('https://www.rightmove.co.uk/properties/', allLinks)

    i = i + 24

# removes empty and repeated links 
cleanLinks = np.char.replace(allLinks, 'https://www.rightmove.co.uk/properties/0#/?channel=RES_BUY', '')
cleanLinks = cleanLinks[cleanLinks != '']
cleanLinks = np.unique(cleanLinks)

print('Number of links acquired: ', len(cleanLinks))
print(cleanLinks)


10  Pages available
Number of links acquired:  219
['https://www.rightmove.co.uk/properties/102227564#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/107409182#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/109937678#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/110514809#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/114573920#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/115021484#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/115645691#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116106038#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116106077#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116108690#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116233709#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116243063#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116491958#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/propertie

Fishing Properties

In [76]:
i = 0
all_info = np.array([])
data = np.array([])

for property in cleanLinks:
    start = time()
    ad = requests.get(property)
    soup = BeautifulSoup(ad.content, "html.parser")
    # extract property data
    propertyAddress = soup.select('h1')[0].text
    propertyType = soup.select('p')[0].text
    typeOfProperty = soup.select('p')[1].text
    noOfBeds = soup.select('p')[2].text
    priceLink = soup.body.find('div', attrs={'class': '_1gfnqJ3Vtd1z40MlC0MzXu'})
    price = priceLink.select('span')[0].text
    newIds = re.findall(r'\b\d+\b', str(cleanLinks[i])) # use new property id
    mapLocation = np.char.add('https://www.rightmove.co.uk/properties/', str(newIds[0]))
    floorPlan = np.char.add(mapLocation, '#/floorplan?activePlan=1&channel=RES_BUY')
    mapLocation = np.char.add(mapLocation, '#/map?channel=RES_BUY')
    
    # filter through the images to find the floor plan link
    floorPlanImage = requests.get(floorPlan)
    floorPlanImage = BeautifulSoup(floorPlanImage.content, "html.parser")
    floorPlanImage = floorPlanImage.find('div', attrs={'id': 'root'})
    for j in range(len(floorPlanImage.select('img'))):
        nextFloorPlanImage = floorPlanImage.select('img')[j].get_attribute_list('src',str)
        nextFloorPlanImage = ' '.join(nextFloorPlanImage)
        if '_max_296x197' in nextFloorPlanImage:
            foundFloorPlanImage = nextFloorPlanImage.replace('_max_296x197', '')
            break
        else:
            foundFloorPlanImage = 'None'
            pass
    
    data = [str(price), str(propertyAddress), str(typeOfProperty), str(noOfBeds), str(foundFloorPlanImage), str(mapLocation)]
    all_info = np.append(all_info, data)
    i = i+1 
    end = time()
    print(str(i)+' out of '+str(len(cleanLinks))+f' It took {round(end - start,2)} seconds!')
    
# split the array into equally divided arrays
all_info = np.split(all_info, len(cleanLinks))
print(all_info)

1 out of 219 It took 2.01 seconds!
2 out of 219 It took 0.79 seconds!
3 out of 219 It took 1.17 seconds!
4 out of 219 It took 1.92 seconds!
5 out of 219 It took 0.94 seconds!
6 out of 219 It took 2.84 seconds!
7 out of 219 It took 0.82 seconds!
8 out of 219 It took 1.79 seconds!
9 out of 219 It took 1.83 seconds!
10 out of 219 It took 1.11 seconds!
11 out of 219 It took 0.86 seconds!
12 out of 219 It took 1.85 seconds!
13 out of 219 It took 1.79 seconds!
14 out of 219 It took 1.88 seconds!
15 out of 219 It took 0.87 seconds!
16 out of 219 It took 1.15 seconds!
17 out of 219 It took 0.81 seconds!
18 out of 219 It took 0.89 seconds!
19 out of 219 It took 0.83 seconds!
20 out of 219 It took 0.81 seconds!
21 out of 219 It took 0.77 seconds!
22 out of 219 It took 0.82 seconds!
23 out of 219 It took 2.15 seconds!
24 out of 219 It took 0.87 seconds!
25 out of 219 It took 0.81 seconds!
26 out of 219 It took 0.83 seconds!
27 out of 219 It took 0.82 seconds!
28 out of 219 It took 2.4 seconds!
29

Image Processing

In [77]:
import importlib 
from numpy import empty
from searchFunction import search 

updated_info = np.array([])

for info in range(len(all_info)):
    print(str(info)+' out of '+str(len(all_info)))
    
    if not all_info[info][4] == 'None':
        # Load the img
        req = urllib.request.urlopen(all_info[info][4])
        arr = np.asarray(bytearray(req.read()))
        img = cv2.imdecode(arr, -1) # 'Load it as it is'

        # Display loaded image
        # cv2.imshow('Floor Plan', img)
        # if cv2.waitKey() & 0xff == 27: quit()

        # OCR
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
        configs = r'--psm 6'
        try:
            rawText = pytesseract.image_to_string(img, config=configs)
        except:
            continue
        rawText = rawText.lower()
        rawText = re.sub(r',', '', str(rawText)) # removes all commas 
        rawText = re.sub(r'\b\.\d+\b', '', str(rawText)) # removes all ints
        #print(rawText)

        newText = search(rawText)
        #print(newText)
        newInfo = list(np.append(all_info[info], newText)) # adding list adds the ',' back into the string array    
        updated_info = np.append(updated_info, newInfo)

# split the array into equally divided arrays and convert to pandas for further processing
updated_info = np.split(updated_info, len(updated_info)/7) 
updated_info = pd.DataFrame(updated_info, columns = ['price', 'propertyAddress', 'typeOfProperty', 'noOfBeds', 'foundFloorPlanImage', 'mapLocation', 'area'])
print(updated_info)

0 out of 219
1 out of 219
710 0 0 0 0
2 out of 219
1124 0 0 0 0
3 out of 219
0 736 0 0 0
4 out of 219
766 0 0 0 0
5 out of 219
815 0 0 0 0
6 out of 219
762 0 0 0 0
7 out of 219
0 0 0 0 0
8 out of 219
0 0 0 0 0
9 out of 219
879 0 0 0 0
10 out of 219
715 0 715 0 0
11 out of 219
0 736 0 0 0
12 out of 219
0 0 0 0 0
13 out of 219
746 0 0 0 0
14 out of 219
735 0 0 0 0
15 out of 219
831 0 0 0 0
16 out of 219
1114 0 1114 0 0
17 out of 219
18 out of 219
861 0 0 0 0
19 out of 219
753 0 0 0 0
20 out of 219
0 0 0 622 622
21 out of 219
684.8 0 0 0 0
22 out of 219
1200 0 0 0 0
23 out of 219
24 out of 219
1247 0 0 0 0
25 out of 219
598 0 0 0 0
26 out of 219
874 0 0 0 0
27 out of 219
678 0 0 0 0
28 out of 219
682 0 0 0 0
29 out of 219
831 0 0 0 0
30 out of 219
736 0 0 0 0
31 out of 219
624 0 0 0 0
32 out of 219
33 out of 219
663 0 0 0 0
34 out of 219
0 0 0 0 804
35 out of 219
971 0 0 0 0
36 out of 219
719 0 0 0 0
37 out of 219
0 689 0 0 0
38 out of 219
676 0 0 0 0
39 out of 219
898 0 0 0 0
40 out of 2

In [82]:
# this should load the old file and append any new data found to it making sure it's all unique data
# and not repeated 
loadData = pd.read_csv(r'output.csv')
frames = [loadData, updated_info]
combinedData = pd.concat(frames)
combinedData = combinedData.drop_duplicates(subset=['foundFloorPlanImage'])

combinedData.to_csv('output.csv', index=False)

combinedData['price'] = combinedData['price'].replace('[\£,]', '', regex=True).astype(int)
combinedData['area'] = combinedData['area'].astype(int)
combinedData['priceRank'] = combinedData['price'].rank(ascending=1)
combinedData['areaRank'] = combinedData['area'].rank(ascending=0)
combinedData['Rank Sum'] = combinedData.priceRank + combinedData.areaRank
combinedData = combinedData.sort_values(by = 'Rank Sum', ascending = True)

combinedData.to_csv('output_processed.csv', index=False)
print(combinedData)

      price                                    propertyAddress typeOfProperty  \
58   112500             Aviation Drive, Colindale, London, NW9           Flat   
67   425000                       West Hendon Broadway, London           Flat   
91   350000              High Mount, Station Road, London, NW4           Flat   
35   375000                 Victoria Road, Hendon, London, NW4      Apartment   
108  459950                              Raffles House, Hendon           Flat   
..      ...                                                ...            ...   
6    615000                            Beaufort Square, London      Apartment   
50   620000                       Beaufort Square, London, NW9           Flat   
174  625000            Heritage Avenue, Colindale, London, NW9           Flat   
45   688000  Meadowlark House\r\nMoorhen Drive off Perryfie...      Apartment   
190  695000                           Southfields, London, NW4  Semi-Detached   

    noOfBeds               